# Libraries

In [ ]:
import pandas as pd
import json
import os
import numpy as np
import collections

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm

# Data

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/SemEval2023/train_all_tasks.csv")

In [ ]:
train_df.groupby('label_sexist').size()

label_sexist
not sexist    10602
sexist         3398
dtype: int64

In [ ]:
train_df.groupby('label_category').size()

label_category
1. threats, plans to harm and incitement      310
2. derogation                                1590
3. animosity                                 1165
4. prejudiced discussions                     333
none                                        10602
dtype: int64

In [ ]:
train_df.groupby('label_vector').size()

label_vector
1.1 threats of harm                                                   56
1.2 incitement and encouragement of harm                             254
2.1 descriptive attacks                                              717
2.2 aggressive and emotive attacks                                   673
2.3 dehumanising attacks & overt sexual objectification              200
3.1 casual use of gendered slurs, profanities, and insults           637
3.2 immutable gender differences and gender stereotypes              417
3.3 backhanded gendered compliments                                   64
3.4 condescending explanations or unwelcome advice                    47
4.1 supporting mistreatment of individual women                       75
4.2 supporting systemic discrimination against women as a group      258
none                                                               10602
dtype: int64

# Preparing Data

In [ ]:
eval_b_labels = pd.read_csv("/content/drive/MyDrive/SemEval2023/dev_task_b_labels.csv")
eval_b_entries = pd.read_csv("/content/drive/MyDrive/SemEval2023/dev_task_b_entries.csv")
eval_c_labels = pd.read_csv("/content/drive/MyDrive/SemEval2023/dev_task_c_labels.csv")
eval_c_entries = pd.read_csv("/content/drive/MyDrive/SemEval2023/dev_task_c_entries.csv")

In [ ]:
train_b_and_c_df = train_df.loc[train_df['label_sexist']!="not sexist"]

In [ ]:
train_b_and_c_df = train_b_and_c_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
eval_b_df = pd.merge(eval_b_entries, eval_b_labels, on = "rewire_id")
eval_c_df = pd.merge(eval_c_entries, eval_c_labels, on = "rewire_id")

In [ ]:
labels_b = eval_b_df['label'].values.tolist()
labels_c = eval_c_df['label'].values.tolist()

# Task A

## Baseline TF-IDF Upsample

In [ ]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Load dataset
data = pd.read_csv("data.csv")
X = data.drop("label", axis=1)
y = data["label"]

# Instanciate a RandomOverSampler object
ros = RandomOverSampler(random_state=0)

# Fit and transform the dataset
X_resampled, y_resampled = ros.fit_resample(X, y)

In [ ]:
len(train_df)

14000

In [ ]:
X = train_df.drop("label_sexist", axis=1)
y = train_df["label_sexist"]

ros = RandomOverSampler(random_state=42)
X_upsampled, y_upsampled = ros.fit_resample(X, y)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_upsampled['text'])
# tfidf_test_vectors = tfidf_vectorizer.transform(test['text'])
tfidf_eval_vectors = tfidf_vectorizer.transform(eval_df['text'])

In [ ]:
svc_classifier = SVC()
svc_classifier.fit(tfidf_train_vectors,y_upsampled)

SVC()

In [ ]:
tfidf_y_pred = svc_classifier.predict(tfidf_eval_vectors)

In [ ]:
ground_truth = pd.read_csv("/content/drive/MyDrive/SemEval2023/df_val.csv")
ground_truth.head()

,rewire_id,text,label
0,sexism2022_english-11999,I do not try to hard. I do what guys who get w...,not sexist
1,sexism2022_english-9895,That sucks. We elected a Navy Seal in Missouri...,not sexist
2,sexism2022_english-11623,Walk away and never look back. If she reaches ...,not sexist
3,sexism2022_english-19035,basically I placed more value on her than I sh...,not sexist
4,sexism2022_english-895,"NYT and Asian ""part ways."" But why on earth wa...",not sexist


In [ ]:
g = ground_truth['label'].values.tolist()
len(g)

2000

In [ ]:
len(tfidf_y_pred)

2000

In [ ]:
print(classification_report(g,tfidf_y_pred))

              precision    recall  f1-score   support

  not sexist       0.84      0.96      0.89      1514
      sexist       0.76      0.43      0.55       486

    accuracy                           0.83      2000
   macro avg       0.80      0.69      0.72      2000
weighted avg       0.82      0.83      0.81      2000



## Baseline TF-IDF Undersample

In [ ]:
X = train_df.drop("label_sexist", axis=1)
y = train_df["label_sexist"]

rus = RandomUnderSampler(random_state=0)
X_undersampled, y_undersampled = rus.fit_resample(X, y)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_undersampled['text'])
tfidf_eval_vectors = tfidf_vectorizer.transform(eval_df['text'])

In [ ]:
svc_classifier = SVC()
svc_classifier.fit(tfidf_train_vectors,y_undersampled)

SVC()

In [ ]:
tfidf_y_pred = svc_classifier.predict(tfidf_eval_vectors)

In [ ]:
print(classification_report(g,tfidf_y_pred))

              precision    recall  f1-score   support

  not sexist       0.88      0.78      0.83      1514
      sexist       0.49      0.66      0.56       486

    accuracy                           0.75      2000
   macro avg       0.69      0.72      0.70      2000
weighted avg       0.78      0.75      0.76      2000



# Task B

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_train_b_vectors = tfidf_vectorizer.fit_transform(train_b_and_c_df['text'])
tfidf_eval_b_vectors = tfidf_vectorizer.transform(eval_b_df['text'])

**SVM**

In [ ]:
svc_classifier = SVC(random_state=42)
svc_classifier.fit(tfidf_train_b_vectors,train_b_and_c_df['label_category'])

SVC(random_state=42)

In [ ]:
svc_tfidf_y_b_pred = svc_classifier.predict(tfidf_eval_b_vectors)

In [ ]:
print(classification_report(labels_b,svc_tfidf_y_b_pred))

                                          precision    recall  f1-score   support

1. threats, plans to harm and incitement       0.83      0.11      0.20        44
                           2. derogation       0.51      0.90      0.65       227
                            3. animosity       0.60      0.29      0.39       167
               4. prejudiced discussions       0.00      0.00      0.00        48

                                accuracy                           0.53       486
                               macro avg       0.49      0.33      0.31       486
                            weighted avg       0.52      0.53      0.46       486



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Logistic Regression**

In [ ]:
# test on train set
# lr_classifier = LogisticRegression(random_state=42).fit(tfidf_train_b_vectors,train_b_and_c_df['label_category'])
# lr_tfidf_y_b_pred = lr_classifier.predict(tfidf_train_b_vectors)
# print(classification_report(train_b_and_c_df['label_category'],lr_tfidf_y_b_pred))

In [ ]:
lr_classifier = LogisticRegression(random_state=42).fit(tfidf_train_b_vectors,train_b_and_c_df['label_category'])

In [ ]:
lr_tfidf_y_b_pred = lr_classifier.predict(tfidf_eval_b_vectors)

In [ ]:
print(classification_report(labels_b,lr_tfidf_y_b_pred))

                                          precision    recall  f1-score   support

1. threats, plans to harm and incitement       0.77      0.23      0.35        44
                           2. derogation       0.56      0.85      0.67       227
                            3. animosity       0.56      0.43      0.48       167
               4. prejudiced discussions       0.67      0.04      0.08        48

                                accuracy                           0.57       486
                               macro avg       0.64      0.38      0.40       486
                            weighted avg       0.59      0.57      0.52       486



# Task C | Independent

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_train_c_vectors = tfidf_vectorizer.fit_transform(train_b_and_c_df['text'])
tfidf_eval_c_vectors = tfidf_vectorizer.transform(eval_c_df['text'])

**SVC**

In [ ]:
svc_classifier.fit(tfidf_train_c_vectors,train_b_and_c_df['label_vector'])

SVC(random_state=42)

In [ ]:
svc_tfidf_y_c_pred = svc_classifier.predict(tfidf_eval_c_vectors)

In [ ]:
print(classification_report(labels_c,svc_tfidf_y_c_pred))

                                                                 precision    recall  f1-score   support

                                            1.1 threats of harm       0.00      0.00      0.00         8
                       1.2 incitement and encouragement of harm       0.67      0.17      0.27        36
                                        2.1 descriptive attacks       0.32      0.80      0.46       102
                             2.2 aggressive and emotive attacks       0.37      0.46      0.41        96
        2.3 dehumanising attacks & overt sexual objectification       0.00      0.00      0.00        29
     3.1 casual use of gendered slurs, profanities, and insults       0.52      0.53      0.52        91
        3.2 immutable gender differences and gender stereotypes       0.40      0.07      0.11        60
                            3.3 backhanded gendered compliments       0.00      0.00      0.00         9
             3.4 condescending explanations or unwelco

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Logistic Regression**

In [ ]:
# test on train
# lr_classifier.fit(tfidf_train_c_vectors,train_b_and_c_df['label_vector'])
# lr_tfidf_y_c_pred = lr_classifier.predict(tfidf_train_c_vectors)
# print(classification_report(train_b_and_c_df['label_vector'],lr_tfidf_y_c_pred))

                                                                 precision    recall  f1-score   support

                                            1.1 threats of harm       0.00      0.00      0.00        56
                       1.2 incitement and encouragement of harm       0.82      0.62      0.71       254
                                        2.1 descriptive attacks       0.60      0.93      0.73       717
                             2.2 aggressive and emotive attacks       0.73      0.87      0.79       673
        2.3 dehumanising attacks & overt sexual objectification       1.00      0.18      0.31       200
     3.1 casual use of gendered slurs, profanities, and insults       0.75      0.90      0.82       637
        3.2 immutable gender differences and gender stereotypes       0.85      0.72      0.78       417
                            3.3 backhanded gendered compliments       0.00      0.00      0.00        64
             3.4 condescending explanations or unwelco

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
lr_classifier.fit(tfidf_train_c_vectors,train_b_and_c_df['label_vector'])
lr_tfidf_y_c_pred = lr_classifier.predict(tfidf_eval_c_vectors)
print(classification_report(labels_c,lr_tfidf_y_c_pred))

                                                                 precision    recall  f1-score   support

                                            1.1 threats of harm       0.00      0.00      0.00         8
                       1.2 incitement and encouragement of harm       0.67      0.33      0.44        36
                                        2.1 descriptive attacks       0.34      0.74      0.47       102
                             2.2 aggressive and emotive attacks       0.39      0.43      0.41        96
        2.3 dehumanising attacks & overt sexual objectification       0.50      0.03      0.06        29
     3.1 casual use of gendered slurs, profanities, and insults       0.50      0.58      0.54        91
        3.2 immutable gender differences and gender stereotypes       0.38      0.20      0.26        60
                            3.3 backhanded gendered compliments       0.00      0.00      0.00         9
             3.4 condescending explanations or unwelco

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Task C | Hierarchical

In [ ]:
grouped = train_b_and_c_df.groupby('label_category')

df_list = []
for group in grouped:
    df = group[1]
    df_list.append(df)

In [ ]:
# category 0 classifier:
tfidf_0_c_vectors = tfidf_vectorizer.transform(df_list[0]['text'])
lr_0 = LogisticRegression(random_state=42)
lr_0.fit(tfidf_0_c_vectors,df_list[0]['label_vector'])

# category 1 classifier:
tfidf_1_c_vectors = tfidf_vectorizer.transform(df_list[1]['text'])
lr_1 = LogisticRegression(random_state=42)
lr_1.fit(tfidf_1_c_vectors,df_list[1]['label_vector'])

# category 2 classifier:
tfidf_2_c_vectors = tfidf_vectorizer.transform(df_list[2]['text'])
lr_2 = LogisticRegression(random_state=42)
lr_2.fit(tfidf_2_c_vectors,df_list[2]['label_vector'])

# category 3 classifier:
tfidf_3_c_vectors = tfidf_vectorizer.transform(df_list[3]['text'])
lr_3 = LogisticRegression(random_state=42)
lr_3.fit(tfidf_3_c_vectors,df_list[3]['label_vector'])


LogisticRegression(random_state=42)

In [ ]:
predictions_df = pd.DataFrame()
predictions_df['rewire_id'] = eval_b_df['rewire_id']
predictions_df['text'] = eval_b_df['text']
predictions_df['label'] = lr_tfidf_y_b_pred

In [ ]:
predictions_df.head()

,rewire_id,text,label
0,sexism2022_english-739,Every woman wants to be a model. It's codeword...,2. derogation
1,sexism2022_english-10787,Poor dude hasn't realized you dont share emoti...,3. animosity
2,sexism2022_english-18547,Can women be loyal? If you make them loyal. Ju...,2. derogation
3,sexism2022_english-6425,Lefties; we haven’t been outraged for an hour....,"1. threats, plans to harm and incitement"
4,sexism2022_english-10001,That slag has some nerve showing her face. Bit...,3. animosity


In [ ]:
c_predictions = []
for row in predictions_df.iterrows():
    vector = tfidf_vectorizer.transform([row[1]['text']])
    if "1" in row[1]['label']:
        prediction = lr_0.predict(vector)
        prediction = prediction[0]
        c_predictions.append(prediction)
    elif "2" in row[1]['label']:
        prediction = lr_1.predict(vector)
        prediction = prediction[0]
        c_predictions.append(prediction)
    elif "3" in row[1]['label']:
        prediction = lr_2.predict(vector)
        prediction = prediction[0]
        c_predictions.append(prediction)
    elif "4" in row[1]['label']:
        prediction = lr_3.predict(vector)
        prediction = prediction[0]
        c_predictions.append(prediction)
    else:
        print("chi?")

In [ ]:
print(classification_report(labels_c,c_predictions))

                                                                 precision    recall  f1-score   support

                                            1.1 threats of harm       0.00      0.00      0.00         8
                       1.2 incitement and encouragement of harm       0.77      0.28      0.41        36
                                        2.1 descriptive attacks       0.36      0.76      0.49       102
                             2.2 aggressive and emotive attacks       0.38      0.49      0.43        96
        2.3 dehumanising attacks & overt sexual objectification       1.00      0.03      0.07        29
     3.1 casual use of gendered slurs, profanities, and insults       0.48      0.51      0.49        91
        3.2 immutable gender differences and gender stereotypes       0.35      0.18      0.24        60
                            3.3 backhanded gendered compliments       0.00      0.00      0.00         9
             3.4 condescending explanations or unwelco

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Assuming the previous step was done right.**

In [ ]:
test_c_predictions = []
for row in eval_b_df.iterrows():
    vector = tfidf_vectorizer.transform([row[1]['text']])
    if "1" in row[1]['label']:
        prediction = lr_0.predict(vector)
        prediction = prediction[0]
        test_c_predictions.append(prediction)
    elif "2" in row[1]['label']:
        prediction = lr_1.predict(vector)
        prediction = prediction[0]
        test_c_predictions.append(prediction)
    elif "3" in row[1]['label']:
        prediction = lr_2.predict(vector)
        prediction = prediction[0]
        test_c_predictions.append(prediction)
    elif "4" in row[1]['label']:
        prediction = lr_3.predict(vector)
        prediction = prediction[0]
        test_c_predictions.append(prediction)
    else:
        print("chi?")
print(classification_report(labels_c,test_c_predictions, zero_division = "warn"))

                                                                 precision    recall  f1-score   support

                                            1.1 threats of harm       0.00      0.00      0.00         8
                       1.2 incitement and encouragement of harm       0.82      1.00      0.90        36
                                        2.1 descriptive attacks       0.60      0.79      0.68       102
                             2.2 aggressive and emotive attacks       0.66      0.61      0.63        96
        2.3 dehumanising attacks & overt sexual objectification       1.00      0.03      0.07        29
     3.1 casual use of gendered slurs, profanities, and insults       0.72      0.92      0.81        91
        3.2 immutable gender differences and gender stereotypes       0.72      0.60      0.65        60
                            3.3 backhanded gendered compliments       0.00      0.00      0.00         9
             3.4 condescending explanations or unwelco

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Task C | Hierarchical | "*Others*" tag added

In [ ]:
grouped = train_b_and_c_df.groupby('label_category')

df_list = []
for group in grouped:
    df = group[1]
    df_list.append(df)

In [ ]:
# grouped_eval = eval_b_df.groupby('label')
# df_eval_list = []
# for group in grouped_eval:
#     df = group[1]
#     df = pd.merge(df, eval_c_df, on = "rewire_id").drop('text_y', axis=1)
#     df = df.rename(columns={'text_x': 'text'})
#     df_eval_list.append(df)

In [ ]:
complement_df_list = []
resample_num = []
for index, df in enumerate(df_list):
    entries_num = df.groupby('label_vector').size()
    average = int(sum(entries_num) / len(entries_num))
    resample_num.append(average)
    complement_df = train_b_and_c_df[~train_b_and_c_df.isin(df).all(1)]
    complement_df_list.append(complement_df)
    random_rows = complement_df.sample(n=average, random_state=42)
    random_rows.loc[:, 'label_vector'] = 'others'
    df = df.append(random_rows, ignore_index=True)
    df_list[index] = df.sample(frac=1, random_state=42)

In [ ]:
# category 0 classifier:
tfidf_0_c_vectors = tfidf_vectorizer.transform(df_list[0]['text'])
lr_0 = LogisticRegression(random_state=42)
lr_0.fit(tfidf_0_c_vectors,df_list[0]['label_vector'])

# category 1 classifier:
tfidf_1_c_vectors = tfidf_vectorizer.transform(df_list[1]['text'])
lr_1 = LogisticRegression(random_state=42)
lr_1.fit(tfidf_1_c_vectors,df_list[1]['label_vector'])

# category 2 classifier:
tfidf_2_c_vectors = tfidf_vectorizer.transform(df_list[2]['text'])
lr_2 = LogisticRegression(random_state=42)
lr_2.fit(tfidf_2_c_vectors,df_list[2]['label_vector'])

# category 3 classifier:
tfidf_3_c_vectors = tfidf_vectorizer.transform(df_list[3]['text'])
lr_3 = LogisticRegression(random_state=42)
lr_3.fit(tfidf_3_c_vectors,df_list[3]['label_vector'])


LogisticRegression(random_state=42)

In [ ]:
predictions_df = pd.DataFrame()
predictions_df['rewire_id'] = eval_b_df['rewire_id']
predictions_df['text'] = eval_b_df['text']
predictions_df['label'] = lr_tfidf_y_b_pred

In [ ]:
predictions_df.head()

,rewire_id,text,label
0,sexism2022_english-739,Every woman wants to be a model. It's codeword...,2. derogation
1,sexism2022_english-10787,Poor dude hasn't realized you dont share emoti...,3. animosity
2,sexism2022_english-18547,Can women be loyal? If you make them loyal. Ju...,2. derogation
3,sexism2022_english-6425,Lefties; we haven’t been outraged for an hour....,"1. threats, plans to harm and incitement"
4,sexism2022_english-10001,That slag has some nerve showing her face. Bit...,3. animosity


In [ ]:
c_predictions = []
other_num = [0, 0, 0, 0]
for row in predictions_df.iterrows():
    vector = tfidf_vectorizer.transform([row[1]['text']])
    if "1" in row[1]['label']:
        prediction = lr_0.predict(vector)
        prediction = prediction[0]
        if prediction == "others":
            other_num[0]+=1
        c_predictions.append(prediction)
    elif "2" in row[1]['label']:
        prediction = lr_1.predict(vector)
        prediction = prediction[0]
        if prediction == "others":
            other_num[1]+=1
        c_predictions.append(prediction)
    elif "3" in row[1]['label']:
        prediction = lr_2.predict(vector)
        prediction = prediction[0]
        if prediction == "others":
            other_num[2]+=1
        c_predictions.append(prediction)
    elif "4" in row[1]['label']:
        prediction = lr_3.predict(vector)
        prediction = prediction[0]
        if prediction == "others":
            other_num[3]+=1
        c_predictions.append(prediction)
    else:
        print("chi?")

In [ ]:
other_num
# len(c_predictions)

[0, 23, 0, 0]

In [ ]:
print(classification_report(labels_c,c_predictions))

                                                                 precision    recall  f1-score   support

                                            1.1 threats of harm       0.00      0.00      0.00         8
                       1.2 incitement and encouragement of harm       0.77      0.28      0.41        36
                                        2.1 descriptive attacks       0.37      0.75      0.50       102
                             2.2 aggressive and emotive attacks       0.40      0.46      0.43        96
        2.3 dehumanising attacks & overt sexual objectification       0.50      0.03      0.06        29
     3.1 casual use of gendered slurs, profanities, and insults       0.48      0.51      0.49        91
        3.2 immutable gender differences and gender stereotypes       0.32      0.17      0.22        60
                            3.3 backhanded gendered compliments       0.00      0.00      0.00         9
             3.4 condescending explanations or unwelco

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1